<a href="https://colab.research.google.com/github/Meghana1999/COVID-19-Data-Analysis/blob/main/Copy_of_COVID_DATA_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data Collection
# Data Precprocessing

# EDA
# Conclusion

In [ ]:
! pip install beautifulsoup4

In [ ]:
! pip install plotly

In [ ]:
from bs4 import BeautifulSoup as soup
from datetime import date, datetime
from urllib.request import Request, urlopen

import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")

 


# Web scraping

In [ ]:
today = datetime.now()
yesterday_str = "%s %d,%d" %(date.today().strftime("%b"), today.day-1, today.year)
yesterday_str

'Feb 2,2022'

In [ ]:
url = "https://www.worldometers.info/coronavirus/#countries"
req = Request(url , headers={'User-Agent':"Mozilla/5.0"})

webpage = urlopen(req)
print(webpage)
page_soup = soup(webpage, "html.parser")

In [ ]:
table = page_soup.findAll("table",{"id":"main_table_countries_yesterday"})

containers = table[0].findAll("tr",{"style":""})
title = containers[0]
del containers[0]

all_data = []
clean = True
for country in containers:
    country_data = []
    country_container = country.findAll("td")
    
    if country_container[1].text == "China":
        continue
    for i in range(1, len(country_container)):
        final_feature = country_container[i].text
        if clean :
            if i !=1 and i != len(country_container)-1:
                final_feature = final_feature.replace(",","")
                
                if final_feature.find('+') != -1:
                    final_feature = final_feature.replace("+","")
                    final_feature = float(final_feature)
                    
                elif final_feature.find('-') != -1:
                    final_feature = final_feature.replace("-","")
                    final_feature = float(final_feature)*-1
                    
        if final_feature == 'N/A':
            final_feature = 0
        elif final_feature == "" or final_feature == " ":
            final_feature = -1
            
        country_data.append(final_feature)
    all_data.append(country_data)

In [ ]:
df = pd.DataFrame(all_data)
df.drop([15,16,17,18,19,20], inplace=True, axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,385170417,3090819.0,5718229,12012.0,305281924,2452000.0,74170264,91837,49414,733.6,-1,-1,-1,All
1,USA,76882290,302177.0,917600,2990.0,47112620,465591.0,28852070,22219,230129,2747,908404627,2719099,334082997,North America
2,India,41803318,172433.0,498987,991.0,39770414,259107.0,1533917,8944,29827,356,732439986,522600,1401531962,Asia
3,Brazil,25813685,188552.0,629078,946.0,22464029,110668.0,2720578,8318,120088,2927,63776166,296692,214957181,South America
4,France,19872989,315363.0,131588,276.0,13108992,304941.0,6632409,3641,303392,2009,231648947,3536476,65502759,Europe


In [ ]:
column_labels = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Serious/Critical","Total cases/1M","Deaths/1M","Total Tests","Test/1M","Population","Continent"]
df.columns = column_labels
df.head()


,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent
0,World,385170417,3090819.0,5718229,12012.0,305281924,2452000.0,74170264,91837,49414,733.6,-1,-1,-1,All
1,USA,76882290,302177.0,917600,2990.0,47112620,465591.0,28852070,22219,230129,2747,908404627,2719099,334082997,North America
2,India,41803318,172433.0,498987,991.0,39770414,259107.0,1533917,8944,29827,356,732439986,522600,1401531962,Asia
3,Brazil,25813685,188552.0,629078,946.0,22464029,110668.0,2720578,8318,120088,2927,63776166,296692,214957181,South America
4,France,19872989,315363.0,131588,276.0,13108992,304941.0,6632409,3641,303392,2009,231648947,3536476,65502759,Europe


In [ ]:
for label in df.columns:
    if label != "Country" and label != "Continent":
        df[label] = pd.to_numeric(df[label])

In [ ]:
df["%Inc Cases"] = df["New Cases"] / df["Total Cases"]*100
df["%Inc Deaths"] = df["New Deaths"] / df["Total Deaths"]*100
df["%Inc Recovered"] = df["New Recovered"] / df["Total Recovered"]*100

In [ ]:
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
0,World,385170417,3090819.0,5718229,12012.0,305281924,2452000.0,74170264.0,91837,49414.0,733.6,-1,-1,-1,All,0.802455,0.210065,0.803192
1,USA,76882290,302177.0,917600,2990.0,47112620,465591.0,28852070.0,22219,230129.0,2747.0,908404627,2719099,334082997,North America,0.393039,0.325850,0.988251
2,India,41803318,172433.0,498987,991.0,39770414,259107.0,1533917.0,8944,29827.0,356.0,732439986,522600,1401531962,Asia,0.412486,0.198602,0.651507
3,Brazil,25813685,188552.0,629078,946.0,22464029,110668.0,2720578.0,8318,120088.0,2927.0,63776166,296692,214957181,South America,0.730434,0.150379,0.492645
4,France,19872989,315363.0,131588,276.0,13108992,304941.0,6632409.0,3641,303392.0,2009.0,231648947,3536476,65502759,Europe,1.586893,0.209746,2.326197


# EDA

In [ ]:
cases = df[["Total Recovered","Active Cases","Total Deaths"]].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df["Total"]/np.sum(cases_df["Total"]),2)
cases_df["Virus"] = ["Covid-19" for i in range(len(cases_df))]
cases_df

,Type,Total,Percentage,Virus
0,Total Recovered,305281924.0,79.26,Covid-19
1,Active Cases,74170264.0,19.26,Covid-19
2,Total Deaths,5718229.0,1.48,Covid-19


In [ ]:
fig = px.bar(cases_df, x="Virus", y="Percentage", color="Type",hover_data=["Total"])
fig.show()

In [ ]:
cases = df[["New Recovered","New Cases","New Deaths"]].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df["Total"]/np.sum(cases_df["Total"]),2)
cases_df["Virus"] = ["Covid-19" for i in range(len(cases_df))]
cases_df

,Type,Total,Percentage,Virus
0,New Recovered,2452000.0,44.14,Covid-19
1,New Cases,3090819.0,55.64,Covid-19
2,New Deaths,12012.0,0.22,Covid-19


In [ ]:
fig = px.bar(cases_df, x="Virus", y="Percentage", color="Type",hover_data=["Total"])
fig.show()

In [ ]:
per = np.round(df[["%Inc Cases","%Inc Deaths","%Inc Recovered"]].loc[0],2)
per_df = pd.DataFrame(per)

per_df.columns = ["Percentage"]

fig = go.Figure()
fig.add_trace(go.Bar(x=per_df.index, y=per_df["Percentage"], marker_color=["Yellow","blue","red"]))



# Continent

In [ ]:
continent_df = df.groupby("Continent").sum().drop("All")
continent_df = continent_df.reset_index()
continent_df

,Continent,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,%Inc Cases,%Inc Deaths,%Inc Recovered
0,Africa,11112919,18056.0,240401,274.0,9942322,15821.0,890255.0,2397,1569912.0,16910.0,93095126,9440379,1391466496,-2.664780,-112.383031,1.766505
1,Asia,101583234,642037.0,1291889,1742.0,93942854,542216.0,6348491.0,26504,3293346.0,27108.0,1670672054,71691027,3223190052,36.581580,-22.432975,22.868245
2,Australia/Oceania,2807316,35263.0,6301,71.0,2450895,43261.0,302020.0,389,587194.0,5000.0,68238624,7719949,42840432,111.811752,405.906937,-9.397037
3,Europe,130227831,1677904.0,1625725,3829.0,98205809,1070544.0,30396296.0,18685,10473779.0,99919.0,1940245883,165524024,748346686,59.894705,101.895126,46.533450
4,North America,90036029,391607.0,1329768,4121.0,58232994,560049.0,30464048.0,28921,5360634.0,45925.0,1017219746,76904205,596434262,22.626798,-261.170083,22.960982
5,South America,49296069,325850.0,1219487,1884.0,40197545,219899.0,4094291.0,14876,1511352.0,29327.0,201934089,11476064,436399556,6.676168,102.797355,8.088038


In [ ]:
def continent_visualization(v_list):
    for label in v_list:
        c_df = continent_df[["Continent",label]]
        c_df["Percentage"]= np.round(100*c_df[label]/np.sum(c_df[label]),2)
        c_df['Virus'] = ["Covid-19" for i in range(len(c_df))]
        
        fig = px.bar(c_df, x="Virus", y="Percentage", color="Continent",hover_data=[label])
        fig.update_layout(title={"text":label})
        fig.show()

In [ ]:
cases_list = ["Total Cases","Active Cases","New Cases","Serious/Critical","Total cases/1M"]

deaths_list = ["Total Deaths","New Deaths","Deaths/1M"]

recovered_list = ["Total Recovered","New Recovered","%Inc Recovered"]

In [ ]:
continent_visualization(cases_list)

# Countries

In [ ]:
df = df.drop([len(df)-1])
country_df = df.drop([0])

country_df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
1,USA,76882290,302177.0,917600,2990.0,47112620,465591.0,28852070.0,22219,230129.0,2747.0,908404627,2719099,334082997,North America,0.393039,0.325850,0.988251
2,India,41803318,172433.0,498987,991.0,39770414,259107.0,1533917.0,8944,29827.0,356.0,732439986,522600,1401531962,Asia,0.412486,0.198602,0.651507
3,Brazil,25813685,188552.0,629078,946.0,22464029,110668.0,2720578.0,8318,120088.0,2927.0,63776166,296692,214957181,South America,0.730434,0.150379,0.492645
4,France,19872989,315363.0,131588,276.0,13108992,304941.0,6632409.0,3641,303392.0,2009.0,231648947,3536476,65502759,Europe,1.586893,0.209746,2.326197
5,UK,17519661,92578.0,157409,534.0,14680867,-1.0,2681385.0,502,255941.0,2300.0,455704373,6657288,68451959,Europe,0.528423,0.339244,-0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Falkland Islands,89,-1.0,-1,-1.0,0,0.0,0.0,-1,24444.0,-1.0,8632,2370777,3641,South America,-1.123596,100.000000,NaN
214,Samoa,33,1.0,-1,-1.0,3,-1.0,30.0,-1,165.0,-1.0,-1,-1,200506,Australia/Oceania,3.030303,100.000000,-33.333333
215,Vatican City,29,-1.0,-1,-1.0,28,-1.0,1.0,-1,36070.0,-1.0,-1,-1,804,Europe,-3.448276,100.000000,-3.571429
216,Western Sahara,10,-1.0,1,-1.0,8,-1.0,1.0,-1,16.0,2.0,-1,-1,620616,Africa,-10.000000,-100.000000,-12.500000


In [ ]:
look =5
country = country_df.columns[1:14]

fig = go.Figure()
c=0
for i in country_df.index:
    if c < look:
        fig.add_trace(go.Bar(name=country_df['Country'][i], x=country, y=country_df.loc[i][1:14]))
    else:
        break
    c +=1
                      
fig.update_layout(title = {"text":f'{look}'})
fig.show()
                      
                      
                      
                      
                      

In [ ]:
look =5
country = country_df.columns[1:14]

fig = go.Figure()
c=0
for i in country_df.index:
    if c < look:
        fig.add_trace(go.Bar(name=country_df['Country'][i], x=country, y=country_df.loc[i][1:14]))
    else:
        break
    c +=1
                      
fig.update_layout(title = {"text":f'top{look} countries affected'}, yaxis_type="log")
fig.show()
                      
                      